In [1]:
from datetime import timedelta
from rich import print
from dynaconf import Dynaconf
from arango import ArangoClient
from utinni import Context
from pathlib import Path

from tiro.plugins.utinni import TiroTSPump
from tiro.plugins.arango import ArangoAgent
from tiro.core import Scenario

conf = Dynaconf(settings_files=["./config/utinni_config.toml"])
scenario = Scenario.from_yaml(Path("./config/scenario.yaml"), Path("./config/use1.yaml"))

In [2]:
context = Context(**conf.on_request)
context.add_pump("tiro", TiroTSPump(scenario=scenario, **conf.pump_init))

In [3]:
table = context.tiro_table(".*Server%(CPU|Memory)Temperature", column="asset_path", only_ts=False, fill_with_default=True)

In [4]:
context.bind(start=-timedelta(hours=1),
             step=timedelta(minutes=10))

# table.value.keys()
table["CPUTemperature"].value

asset_path,Room.6e4ed6b6-cacf-11ec-9bb1-0242c0a8f005.Rack.6e4ee5f2-cacf-11ec-9bb1-0242c0a8f005.Server.6e4ee782-cacf-11ec-9bb1-0242c0a8f005.Telemetry.CPUTemperature,Room.6e4ed6b6-cacf-11ec-9bb1-0242c0a8f005.Rack.6e4ee5f2-cacf-11ec-9bb1-0242c0a8f005.Server.6e4ee82c-cacf-11ec-9bb1-0242c0a8f005.Telemetry.CPUTemperature,Room.6e4ed6b6-cacf-11ec-9bb1-0242c0a8f005.Rack.6e4ee5f2-cacf-11ec-9bb1-0242c0a8f005.Server.6e4eea2a-cacf-11ec-9bb1-0242c0a8f005.Telemetry.CPUTemperature,Room.6e4ed6b6-cacf-11ec-9bb1-0242c0a8f005.Rack.6e4ee5f2-cacf-11ec-9bb1-0242c0a8f005.Server.6e4eebd8-cacf-11ec-9bb1-0242c0a8f005.Telemetry.CPUTemperature,Room.6e4ed6b6-cacf-11ec-9bb1-0242c0a8f005.Rack.6e4ee5f2-cacf-11ec-9bb1-0242c0a8f005.Server.6e4eec5a-cacf-11ec-9bb1-0242c0a8f005.Telemetry.CPUTemperature,Room.6e4ed6b6-cacf-11ec-9bb1-0242c0a8f005.Rack.6e4ee5f2-cacf-11ec-9bb1-0242c0a8f005.Server.6e4eecbe-cacf-11ec-9bb1-0242c0a8f005.Telemetry.CPUTemperature,Room.6e4ed6b6-cacf-11ec-9bb1-0242c0a8f005.Rack.6e4ee5f2-cacf-11ec-9bb1-0242c0a8f005.Server.6e4eed2c-cacf-11ec-9bb1-0242c0a8f005.Telemetry.CPUTemperature,Room.6e4ed6b6-cacf-11ec-9bb1-0242c0a8f005.Rack.6e4ee5f2-cacf-11ec-9bb1-0242c0a8f005.Server.6e4eed90-cacf-11ec-9bb1-0242c0a8f005.Telemetry.CPUTemperature,Room.6e4ed6b6-cacf-11ec-9bb1-0242c0a8f005.Rack.6e4eedfe-cacf-11ec-9bb1-0242c0a8f005.Server.6e4eeea8-cacf-11ec-9bb1-0242c0a8f005.Telemetry.CPUTemperature,Room.6e4ed6b6-cacf-11ec-9bb1-0242c0a8f005.Rack.6e4eedfe-cacf-11ec-9bb1-0242c0a8f005.Server.6e4eef98-cacf-11ec-9bb1-0242c0a8f005.Telemetry.CPUTemperature,...,Room.be9cff14-cb31-11ec-9a6f-0242ac120005.Rack.be9d5dd8-cb31-11ec-9a6f-0242ac120005.Server.be9d626a-cb31-11ec-9a6f-0242ac120005.Telemetry.CPUTemperature,Room.be9cff14-cb31-11ec-9a6f-0242ac120005.Rack.be9d5dd8-cb31-11ec-9a6f-0242ac120005.Server.be9d62ce-cb31-11ec-9a6f-0242ac120005.Telemetry.CPUTemperature,Room.be9cff14-cb31-11ec-9a6f-0242ac120005.Rack.be9d5dd8-cb31-11ec-9a6f-0242ac120005.Server.be9d6332-cb31-11ec-9a6f-0242ac120005.Telemetry.CPUTemperature,Room.be9cff14-cb31-11ec-9a6f-0242ac120005.Rack.be9d5dd8-cb31-11ec-9a6f-0242ac120005.Server.be9d63b4-cb31-11ec-9a6f-0242ac120005.Telemetry.CPUTemperature,Room.be9cff14-cb31-11ec-9a6f-0242ac120005.Rack.be9d5dd8-cb31-11ec-9a6f-0242ac120005.Server.be9d6418-cb31-11ec-9a6f-0242ac120005.Telemetry.CPUTemperature,Room.be9cff14-cb31-11ec-9a6f-0242ac120005.Server.be9d649a-cb31-11ec-9a6f-0242ac120005.Telemetry.CPUTemperature,Room.be9cff14-cb31-11ec-9a6f-0242ac120005.Server.be9d6512-cb31-11ec-9a6f-0242ac120005.Telemetry.CPUTemperature,Room.be9cff14-cb31-11ec-9a6f-0242ac120005.Server.be9d6634-cb31-11ec-9a6f-0242ac120005.Telemetry.CPUTemperature,Room.be9cff14-cb31-11ec-9a6f-0242ac120005.Server.be9d6698-cb31-11ec-9a6f-0242ac120005.Telemetry.CPUTemperature,Room.be9cff14-cb31-11ec-9a6f-0242ac120005.Server.be9d66fc-cb31-11ec-9a6f-0242ac120005.Telemetry.CPUTemperature
2022-05-03 23:00:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-03 23:10:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-03 23:20:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-03 23:30:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-03 23:40:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-03 23:50:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-04 00:00:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0


In [5]:
context.tiro_table(".*Site", column="Room", only_ts=False, fill_with_default=True, agg_fn="first").value

{'Site': Room                      be9cff14-cb31-11ec-9a6f-0242ac120005
 2022-05-03 23:00:00+00:00                           Hunt-Roman
 2022-05-03 23:10:00+00:00                           Hunt-Roman
 2022-05-03 23:20:00+00:00                           Hunt-Roman
 2022-05-03 23:30:00+00:00                           Hunt-Roman
 2022-05-03 23:40:00+00:00                           Hunt-Roman
 2022-05-03 23:50:00+00:00                           Hunt-Roman
 2022-05-04 00:00:00+00:00                           Hunt-Roman}

In [6]:
data = context.tiro_pump.capture_status(flatten=True, fill_with_default=True, skip_telemetry_in_tsdb=True)
print(data)

{
    'Room.be9cff14-cb31-11ec-9a6f-0242ac120005.Attribute.Site': {
        'value': 'Hunt-Roman',
        'unit': None,
        'timestamp': '2022-05-03T22:38:55.740912'
    },
    'Room.b9c89ae2-c984-11ec-883c-0242c0a8c004.Rack.b9c89c2c-c984-11ec-883c-0242c0a8c004.Serv
er.b9c89dd0-c984-11ec-883c-0242c0a8c004.Telemetry.CPUTemperature': {
        'value': 20,
        'timestamp': datetime.datetime(2022, 5, 4, 0, 6, 10, 390288),
        'unit': '°C'
    },
    'Room.b9c89ae2-c984-11ec-883c-0242c0a8c004.Rack.b9c89c2c-c984-11ec-883c-0242c0a8c004.Serv
er.b9c89e70-c984-11ec-883c-0242c0a8c004.Telemetry.CPUTemperature': {
        'value': 20,
        'timestamp': datetime.datetime(2022, 5, 4, 0, 6, 10, 390313),
        'unit': '°C'
    },
    'Room.b9c89ae2-c984-11ec-883c-0242c0a8c004.Rack.b9c89c2c-c984-11ec-883c-0242c0a8c004.Serv
er.b9c89ed4-c984-11ec-883c-0242c0a8c004.Telemetry.CPUTemperature': {
        'value': 20,
        'timestamp': datetime.datetime(2022, 5, 4, 0, 6, 10, 390326),
        'unit': '°C'
    },
    'Room.b9c89ae2-c984-11ec-883c-0242c0a8c004.Rack.b9c89f42-c984-11ec-883c-0242c0a8c004.Serv
er.b9c89fce-c984-11ec-883c-0242c0a8c004.Telemetry.CPUTemperature': {
        'value': 20,
        'timestamp': datetime.datetime(2022, 5, 4, 0, 6, 10, 390336),
        'unit': '°C'
    },
    'Room.b9c89ae2-c984-11ec-883c-0242c0a8c004.Rack.b9c89f42-c984-11ec-883c-0242c0a8c004.Serv
er.b9c8a032-c984-11ec-883c-0242c0a8c004.Telemetry.CPUTemperature': {
        'value': 20,
        'timestamp': datetime.datetime(2022, 5, 4, 0, 6, 10, 390346),
        'unit': '°C'
    },
    'Room.b9c89ae2-c984-11ec-883c-0242c0a8c004.Rack.b9c8a08c-c984-11ec-883c-0242c0a8c004.Serv
er.b9c8a0f0-c984-11ec-883c-0242c0a8c004.Telemetry.CPUTemperature': {
        'value': 20,
        'timestamp': datetime.datetime(2022, 5, 4, 0, 6, 10, 390355),
        'unit': '°C'
    },
    'Room.b9c89ae2-c984-11ec-883c-0242c0a8c004.Rack.b9c8a08c-c984-11ec-883c-0242c0a8c004.Serv
er.b9c8a154-c984-11ec-883c-0242c0a8c004.Telemetry.CPUTemperature': {
        'value': 20,
        'timestamp': datetime.datetime(2022, 5, 4, 0, 6, 10, 390363),
        'unit': '°C'
    },
    'Room.b9c89ae2-c984-11ec-883c-0242c0a8c004.Rack.b9c8a08c-c984-11ec-883c-0242c0a8c004.Serv
er.b9c8a1ae-c984-11ec-883c-0242c0a8c004.Telemetry.CPUTemperature': {
        'value': 20,
        'timestamp': datetime.datetime(2022, 5, 4, 0, 6, 10, 390372),
        'unit': '°C'
    },
    'Room.b9c89ae2-c984-11ec-883c-0242c0a8c004.Rack.b9c8a08c-c984-11ec-883c-0242c0a8c004.Serv
er.b9c8a208-c984-11ec-883c-0242c0a8c004.Telemetry.CPUTemperature': {
        'value': 20,
        'timestamp': datetime.datetime(2022, 5, 4, 0, 6, 10, 390380),
        'unit': '°C'
    },
    'Room.b9c89ae2-c984-11ec-883c-0242c0a8c004.Rack.b9c8a08c-c984-11ec-883c-0242c0a8c004.Serv
er.b9c8a258-c984-11ec-883c-0242c0a8c004.Telemetry.CPUTemperature': {
        'value': 20,
        'timestamp': datetime.datetime(2022, 5, 4, 0, 6, 10, 390389),
        'unit': '°C'
    },
    'Room.b9c89ae2-c984-11ec-883c-0242c0a8c004.Rack.b9c8a08c-c984-11ec-883c-0242c0a8c004.Serv
er.b9c8a2d0-c984-11ec-883c-0242c0a8c004.Telemetry.CPUTemperature': {
        'value': 20,
        'timestamp': datetime.datetime(2022, 5, 4, 0, 6, 10, 390398),
        'unit': '°C'
    },
    'Room.b9c89ae2-c984-11ec-883c-0242c0a8c004.Rack.b9c8a08c-c984-11ec-883c-0242c0a8c004.Serv
er.b9c8a32a-c984-11ec-883c-0242c0a8c004.Telemetry.CPUTemperature': {
        'value': 20,
        'timestamp': datetime.datetime(2022, 5, 4, 0, 6, 10, 390406),
        'unit': '°C'
    },
    'Room.b9c89ae2-c984-11ec-883c-0242c0a8c004.Rack.b9c8a08c-c984-11ec-883c-0242c0a8c004.Serv
er.b9c8a38e-c984-11ec-883c-0242c0a8c004.Telemetry.CPUTemperature': {
        'value': 20,
        'timestamp': datetime.datetime(2022, 5, 4, 0, 6, 10, 390414),
        'unit': '°C'
    },
    'Room.b9c89ae2-c984-11ec-883c-0242c0a8c004.Rack.b9c8a08c-c984-11ec-883c-0242c0a8c004.Serv
er.b9c8a3e8-c984-11ec-883c-0242c0a8c004.Telemetry.CPUTemperatu

In [7]:
context.tiro_pump.status_as_df(fill_with_default=False)

,asset_path,type,field,value,unit
0,Room.b9c89ae2-c984-11ec-883c-0242c0a8c004,Telemetry,Temperature,26.45,°C
1,Room.b9c89ae2-c984-11ec-883c-0242c0a8c004.Rack...,Telemetry,BackTemperature,21.96,°C
2,Room.b9c89ae2-c984-11ec-883c-0242c0a8c004.Rack...,Telemetry,MemoryTemperature,14.93,°C
3,Room.b9c89ae2-c984-11ec-883c-0242c0a8c004.Rack...,Telemetry,MemoryTemperature,16.35,°C
4,Room.b9c89ae2-c984-11ec-883c-0242c0a8c004.Rack...,Telemetry,MemoryTemperature,26.94,°C
...,...,...,...,...,...
327,Room.be9cff14-cb31-11ec-9a6f-0242ac120005.Rack...,Telemetry,MemoryTemperature,18.2,°C
328,Room.be9cff14-cb31-11ec-9a6f-0242ac120005.Rack...,Telemetry,MemoryTemperature,18.84,°C
329,Room.be9cff14-cb31-11ec-9a6f-0242ac120005.Rack...,Telemetry,MemoryTemperature,14.28,°C
330,Room.be9cff14-cb31-11ec-9a6f-0242ac120005.Rack...,Telemetry,MemoryTemperature,15.1,°C
